In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
from google.colab import files
import gspread
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe
import numpy as np
import pandas as pd
from dateutil.parser import parse

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Bulan Sebelumnya
date_start_prev = '2025-06-01'
date_end_prev = '2025-06-30'
# Bulan Berjalan
date_start = '2025-05-01'
date_end = '2025-05-31'
# Bulan lalu
month_prev = 'May'
month = 'June'

In [ ]:
# Read Sheet Armada
spreadsheet_monitoring_armada = gc.open("MONITORING ARMADA 2025 (Responses)")
sheet_monitoring = spreadsheet_monitoring_armada.worksheet("Form Responses 1")
df_monitoring = get_as_dataframe(sheet_monitoring, evaluate_formulas=True)
non_vehicle_cols = ['Timestamp', 'Bulan', 'PMA']
len_df_monitoring = len(df_monitoring)

vehicle_base = [
    'Detail', 'Channel', 'Divisi', 'Tipe Fisik Armada', 'NOPOL', 'Unit', 'Vendor',
    'Biaya Sewa Kendaraan Niaga (By FBL3N)', 'Tahun Kendaraan', 'Masa Berlaku STNK',
    'PIC User', 'NIK PIC', 'Jabatan', 'Apakah masih ada kendaraan yang akan di data?'
]

vehicles = []

for i in range(50):  # 0 to 49
    cols = [f"{col}" if i == 0 else f"{col}.{i}" for col in vehicle_base]
    sub_df = df_monitoring[non_vehicle_cols + cols].copy()
    sub_df.columns = non_vehicle_cols + vehicle_base
    sub_df['Kendaraan_Ke'] = i + 1
    vehicles.append(sub_df)
df_long = pd.concat(vehicles, ignore_index=True)
df_long = df_long[df_long['NOPOL'].notna()]
df_armada = df_long.sort_values(['Timestamp', 'Kendaraan_Ke'])

sheet_to_upload= gc.open("Armada").worksheet("Sheet1")
sheet_to_upload.clear()
set_with_dataframe(sheet_to_upload, df_armada)

# Read Spreadsheet - MONITORING VEHICLE DAILY (Responses)
spreadsheet_responses = gc.open("MONITORING VEHICLE DAILY (Responses)")
sheet_responses = spreadsheet_responses.worksheet("Form Responses 1")
df_responses = get_as_dataframe(sheet_responses, evaluate_formulas=True)
# Read Spreadsheet - MAPPING KODE DEPO
spreadsheet_kode_depo = gc.open("MAPPING KODE DEPO")
sheet_kode_depo = spreadsheet_kode_depo.worksheet("Sheet1")
df_kode_depo = get_as_dataframe(sheet_kode_depo, evaluate_formulas=True)
df_responses = df_responses[['REGION','Jenis Kendaraan',
       'Nomor Kendaraan', 'Nilai Pengisian BBM (Rupiah)', 'Jenis BBM', 'Liter BBM',
       'KM Terakhir Kendaraan (ketika pengisian)',
       'Hari/Tanggal Pengisian BBM','DEPO']]

# Menambah Kolom Nama Depo
df_responses.loc[:, ['Nama Depo']] = df_responses['DEPO'].str[8:]

# Menambah Kolom Kode Depo
df_responses.loc[:, ['Kode Depo']] = df_responses['DEPO'].str[:7]
df_responses.loc[:, ['Kode Depo']] = pd.to_numeric(df_responses['Kode Depo'], errors='coerce').astype('Int64')

# Join dengan budget berdasarkan Dapul/Lapul
df_kode_depo = df_kode_depo[['KODE DIST','StatusDist', 'Budget']]
merged = pd.merge(
      df_responses,
      df_kode_depo,
      left_on=['Kode Depo'],
      right_on=['KODE DIST'],
      how='left'
    )

# Mengubah kolom budget menjadi float
merged.loc[:, ['Budget']]= pd.to_numeric(merged['Budget'], errors='coerce').astype('float64')

# Mengubah Hari/Tanggal Pengisian BBM menjadi date
merged['Hari/Tanggal Pengisian BBM'] = pd.to_datetime(merged['Hari/Tanggal Pengisian BBM'], errors='coerce')

# Menambah kolom liter bbm, hari pengisian, dan km terakhir sebelumnya
merged = merged.sort_values(['Nomor Kendaraan', 'Hari/Tanggal Pengisian BBM'])
merged['LITER BBM PREV'] = merged.groupby('Nomor Kendaraan')['Liter BBM'].shift(1)
merged['HARI PENGISIAN PREV'] = merged.groupby('Nomor Kendaraan')['Hari/Tanggal Pengisian BBM'].shift(1)
merged['KM TERAKHIR PREV'] = merged.groupby('Nomor Kendaraan')['KM Terakhir Kendaraan (ketika pengisian)'].shift(1)

# Mengubah hari pengisian prev manjadi date
merged['HARI PENGISIAN PREV'] = pd.to_datetime(merged['HARI PENGISIAN PREV'], errors='coerce')

# Menambah kolom selisih hari
# Selisih hari adalah tanggal pengisian saat ini kurang tanggal pengisian sebelumnya
merged['SELISIH HARI'] = (merged['Hari/Tanggal Pengisian BBM'] - merged['HARI PENGISIAN PREV']).dt.days

# Menambah kolom BBM per harian
# BBM per harian adalah jumlah liter sebelumnya kurang selisih hari
merged['BBM DAY'] = merged['LITER BBM PREV']/merged['SELISIH HARI']

# mengubah kolom invalid menjadi nan
# case kolom invalid adalah karena kolom DEPO kosong atau invalid
merged.replace([np.inf, -np.inf], np.nan, inplace=True)

sheetname = ['Monitoring BBM-All', 'Monitoring BBM', 'Monitoring BBM-Bulan Berjalan']
for name in sheetname:
  if name == 'Monitoring BBM':
    filter_data = merged[(merged['Hari/Tanggal Pengisian BBM'] >= pd.to_datetime(date_start_prev)) & (merged['Hari/Tanggal Pengisian BBM'] <= pd.to_datetime(date_end_prev))]
  elif name == 'Monitoring BBM-Bulan Berjalan':
    filter_data = merged[(merged['Hari/Tanggal Pengisian BBM'] >= pd.to_datetime(date_start)) & (merged['Hari/Tanggal Pengisian BBM'] <= pd.to_datetime(date_end))]
  else:
    filter_data = merged

  # Menambah kolom BBM DAY BY Nomor Kendaraan
  # BBM per harian adalah jumlah liter sebelumnya kurang selisih hari
  # merged['BBM DAY BY Nomor Kendaraan'] = merged.groupby('Nomor Kendaraan')['BBM DAY'].transform('mean')
  avg_bbm = filter_data.dropna(subset=['BBM DAY']).groupby('Nomor Kendaraan')['BBM DAY'].mean()
  filter_data['BBM DAY BY Nomor Kendaraan'] = filter_data['Nomor Kendaraan'].map(avg_bbm)

  # Menambah kolom jenis BBM fix
  # Jenis BBM fix adalah pertalite, dexlite, solar, dan pertamax, selain itu dikategorikan other
  fuel_types = ["Pertalite", "Dexlite", "Solar", "Pertamax"]
  filter_data['JENIS BBM FIX'] = np.where(filter_data['Jenis BBM'].isin(fuel_types), filter_data['Jenis BBM'], "Other")

  # menambah kolom on/over budget
  # on budget jika nilai BBM lebih kecil sama dengan dari budget
  # over budget jika nilai BBM lebih besar dari budget
  # jika kolom bbm day atau budget kosong maka kolom ON/OVER Budget juga kosong
  filter_data['ON/OVER BUDGET'] = np.where(
      np.logical_or(filter_data['BBM DAY'].isna(), filter_data['Budget'].isna() ), '',
      np.where(filter_data['BBM DAY'] > filter_data['Budget'], 'OVER BUDGET','ON BUDGET')
  )

  # menambah kolom percentage
  # kolom percentage adalah (bbm per hari by nomor kendaraan - budget)/budget
  # jika kolom bbm day atau budget kosong maka percetage menjadi -1
  filter_data['PERCENTAGE'] = np.where(
      np.logical_or(filter_data['BBM DAY BY Nomor Kendaraan'].isna(), filter_data['Budget'].isna() ), -1,
      np.where(
      (filter_data['BBM DAY BY Nomor Kendaraan'] - filter_data['Budget']) / filter_data['Budget'] > 0,
      (filter_data['BBM DAY BY Nomor Kendaraan'] - filter_data['Budget']) / filter_data['Budget'],
      0
  ))

  # Menambah kolom category
  # category adalah kolom yang berisi kategori dari percentage
  # jika kolom percentage kosong atau -1 maka kolom category akan kosong
  conditions = [
      filter_data['PERCENTAGE'].isna() | (filter_data['PERCENTAGE'] == '')| (filter_data['PERCENTAGE'] == -1),
      filter_data['PERCENTAGE'] == 0,
      (filter_data['PERCENTAGE'] > 0) & (filter_data['PERCENTAGE'] <= 0.10),
      (filter_data['PERCENTAGE'] > 0.10) & (filter_data['PERCENTAGE'] <= 0.20),
      (filter_data['PERCENTAGE'] > 0.20) & (filter_data['PERCENTAGE'] <= 0.30),
      (filter_data['PERCENTAGE'] > 0.30) & (filter_data['PERCENTAGE'] <= 0.40),
      (filter_data['PERCENTAGE'] > 0.40) & (filter_data['PERCENTAGE'] <= 0.50),
      (filter_data['PERCENTAGE'] > 0.50) & (filter_data['PERCENTAGE'] <= 1.00),
      (filter_data['PERCENTAGE'] > 1.00)]
  choices = ['','ON BUDGET','0-10%','10-20%','20-30%','30-40%','40-50%','50-100%','>100%']
  filter_data['Category'] = np.select(conditions, choices, default='')

  filter_data['Category2'] = np.where(
      np.logical_or(filter_data['PERCENTAGE'].isna(), filter_data['PERCENTAGE']=='',filter_data['PERCENTAGE']==-1  ), '',
      np.where(filter_data['PERCENTAGE'] == 0, 'ON BUDGET','OVER BUDGET')
  )

  # Menambah kolom selisih km
  # Selisih km adalah selisih km terakhir saat ini dengan yang sebelumnya
  # jika km terakhir sebelumnya kosong makan selisih km juga kosong
  filter_data.loc[:, ['KM Terakhir Kendaraan (ketika pengisian)']]= pd.to_numeric(filter_data['KM Terakhir Kendaraan (ketika pengisian)'], errors='coerce').astype('float64')
  filter_data.loc[:, ['KM TERAKHIR PREV']]= pd.to_numeric(filter_data['KM TERAKHIR PREV'], errors='coerce').astype('float64')
  filter_data['SELISIH KM'] = np.where(filter_data['KM TERAKHIR PREV'].isna(), np.nan, filter_data['KM Terakhir Kendaraan (ketika pengisian)'] -filter_data['KM TERAKHIR PREV'])

  # Menambah kolom rasio
  # rasio adalah selisih km/liter bbm sebelumnya
  filter_data['RASIO'] = filter_data['SELISIH KM'] / filter_data['LITER BBM PREV']

  # Menambah kolom harga bbm/liter
  # harga bbm/liter adalah nilai perngisian bbm(rupiah) per liter bbm
  filter_data['Harga BBM/Liter'] = filter_data['Nilai Pengisian BBM (Rupiah)']/filter_data['Liter BBM']

  # Menambah kolom CEK
  # Jika kolom harga bbm/liter kurang dari 5000 dab lebih dari 20000 makan kolom CEK akan berisi CEK
  # Selain itu kolom CEK akan berisi kosong
  conditions2 = [
      filter_data['Harga BBM/Liter'] < 5000,
      filter_data['Harga BBM/Liter'] > 20000]
  choices2 = ['Harga BBM/Liter < 5000','Harga BBM/Liter > 20000',]
  filter_data['CEK1'] = np.select(conditions2, choices2, default='')

  # Menambah kolom CEK2
  # Jika kolom selisih km <= 0 maka kolom CEK2 akan berisi CEK selain itu kosong
  filter_data['CEK2'] = np.where(filter_data['SELISIH KM'] <= 0,'KM Terakhir sebelumnya ' + filter_data['KM TERAKHIR PREV'].astype(str),'')

  # Menambah kolom CEK3
  # Jika nomor kendaraan tidak ada di sheet armada maka akan berisi CEK selain itu kosong
  # if name == 'Monitoring BBM':
  #   df_armada =  df_armada[df_armada['Bulan'].str.lower() == month_prev]
  # elif name == 'Monitoring BBM-Bulan Berjalan':
  #   df_armada =  df_armada[df_armada['Bulan'].str.lower() == month]
  filter_data['CEK3'] = np.where(filter_data['Nomor Kendaraan'].isin(df_armada['NOPOL']), '', 'Nomor kendaaraan tidak terdapat di Monitoring Armada 2025')

  # Menambah kolom CEK4
  # Jika rasio >20 berisi CEK selain itu kosong
  filter_data['CEK4'] = np.where(filter_data['RASIO'] > 20, 'Rasio > 20, KM Terakhir sebelumnya '+ filter_data['KM TERAKHIR PREV'].astype(str)+', liter sebelumnya '+filter_data['LITER BBM PREV'].astype(str), '')

  # Menambah kolom CEK5
  # Jika kolom Selisih KM negatif maka KM Terakhir yang sebelumnya juga ditandai untuk di cek
  filter_data['CEK5'] = filter_data.groupby('Nomor Kendaraan')['CEK2'].shift(-1)
  filter_data['CEK5'] = np.where(filter_data['CEK5'].notna() & (filter_data['CEK5'] != ''), 'KM Terakhir lebih besar dari pengisian berikutnya', '')

  # merged['LITER BBM PREV'] = merged.groupby('Nomor Kendaraan')['Liter BBM'].shift(1)

  # Menambah kolom CEK ALL
  # Jika kolom CEK atau CEK2 atau CEK3 atau CEK4 berisi CEK maka kolom CEK ALL akan berisi CEK selain itu kosong
  filter_data['CEK ALL'] = np.where((filter_data['CEK1'] != '') | (filter_data['CEK2'] != '') | (filter_data['CEK3'] != '') | (filter_data['CEK4'] != '')| (filter_data['CEK5'] != ''),'CEK','')

  # Menambah kolom note
  filter_data['NOTE'] = filter_data.apply(
    lambda row: ','.join([str(row[col]) for col in ['CEK1', 'CEK2', 'CEK3', 'CEK4', 'CEK5'] if pd.notna(row[col]) and row[col] != '']),
    axis=1
)


  # Cek ulang, mengubah kolom yang invalid jadi nan
  filter_data.replace([np.inf, -np.inf], np.nan, inplace=True)
  filter_data = filter_data.sort_values(['REGION', 'DEPO','Nomor Kendaraan', 'Hari/Tanggal Pengisian BBM'])

  # Upload All
  sheet_to_upload= gc.open(name).worksheet("Sheet1")
  sheet_to_upload.clear()
  set_with_dataframe(sheet_to_upload, filter_data)